# Simple bot

In [6]:
#!pip install pymorphy2[fast]

In [200]:
import string
import nltk
import annoy
import numpy as np
import tensorflow as tf
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm import tqdm_notebook
from transformers import TFAutoModel, AutoTokenizer

In [199]:
!head -3 prepared_answers.txt

	вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .	меня вобще прикалывает эта тема :). 
Что делать, сегодня нашёл 2 миллиона рублей? .	Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. 


In [17]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [74]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [31]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [36]:
bert = TFAutoModel.from_pretrained("Geotrend/bert-base-ru-cased")
tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-ru-cased")

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [42]:
q_len = []
a_len = []
with open("prepared_answers.txt", "r") as f:
    counter = 0
    for line in tqdm_notebook(f):
        spls = line.split("\t")
        q_len.append(len(spls[0].split()))
        a_len.append(len(spls[1].split()))

<ipython-input-42-a19036b625f7>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


In [47]:
np.median(q_len), np.mean(q_len), np.quantile(q_len, 0.75), np.quantile(q_len, 0.85), np.quantile(q_len, 0.95), np.max(q_len)

(12.0, 20.454707214215595, 21.0, 31.0, 63.0, 862)

In [163]:
bert_index = annoy.AnnoyIndex(768 , 'angular')

index_map = {}
counter = 0
with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 70:
            continue
        
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
             
        out_state = bert(**tok)[1].numpy()[0]
        bert_index.add_item(counter, out_state)
        counter += 1
        
        if counter > 20000:
            break

bert_index.build(10)

<ipython-input-163-6fb6757690bf>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


True

In [205]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    vector = bert(**tok)[1].numpy()[0]
    answers = index.get_nns_by_vector(vector, 1)
    return [index_map[i] for i in answers]

In [208]:
get_response("какой навигатор выбрать?", bert_index_out, index_map)

['Если Вы планируете путешествовать по России - то NAVITEL. У нее мощные карты, которые честно поддерживаются в актуальном состоянии. Если по Европе - IGO и TOMTOM. Но они частенько врут о состоянии пробок и пытаются переложить маршрут, объезжая затор, которого по факту нет.. \n']

In [211]:
get_response("какие новые фильмы есть?", bert_index_out, index_map)

['<p>«Принц» (англ.\xa0The Prince)\xa0— кинофильм, криминальный триллер режиссёра Брайана А. Миллера. Совместное производство США и Великобритания, 2014 год. </p> <h2>Сюжет</h2> <p>Скромный автомеханик Пол (Джейсон Патрик) из Миссисипи\xa0— мужчина средних лет, тщательно скрывающий своё прошлое. Его дочь Бэт (Джина Мантенья), девушка старшего школьного возраста, неожиданно исчезает. Пол отправляется на её поиски. К нему присоединяется подруга Бэт Анджела (Джессика Лаундс). Выясняется, что девушка была связана с наркодилером по кличке «Фармацевт» (Кёртис Джеймс). Дальнейшие поиски ведут в Новый Орлеан к местному криминальному авторитету Омару (Брюс Уиллис)\xa0— старому врагу главного героя. Двадцать лет назад, как выясняется, Пол был сотрудником спецслужбы и в ходе проведения боевой операции стал виновником гибели жены и маленькой дочери Омара. </p> <h2>В ролях</h2> <ul> <li>Джейсон Патрик\xa0— Пол</li> <li>Брюс Уиллис\xa0— Омар</li> <li>Джон Кьюсак\xa0— Сэм</li> <li>Кёртис Джексон (50 

In [212]:
get_response("какие новые фильмы посмотреть?", bert_index_out, index_map)

['а нету и смотреть нефиг ((((((. \n']

In [219]:
get_response("куда сходить на юге Москвы?", bert_index_out, index_map)

['В зоопарк. С завтрашнего дня часть музеев и некоторые сеансы в кино.. \n']